In [2]:
pip install scikit-surprise
pip install jsonlines

     ---------------------------------------- 0.0/772.0 kB ? eta -:--:--
     - -------------------------------------- 30.7/772.0 kB ? eta -:--:--
     ----- -------------------------------- 112.6/772.0 kB 1.7 MB/s eta 0:00:01
     ------------ ------------------------- 245.8/772.0 kB 1.9 MB/s eta 0:00:01
     ----------------- -------------------- 348.2/772.0 kB 2.0 MB/s eta 0:00:01
     ----------------- -------------------- 348.2/772.0 kB 2.0 MB/s eta 0:00:01
     ------------------- ------------------ 399.4/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ----------------------------------- -- 716.8/772.0 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 772.0/772.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing 

In [1]:
import pandas as pd
import json
import gzip
import time
# Load the data
df1 = pd.read_csv('df_ca.csv')
df1 = df1.drop('Unnamed: 0', axis=1)
df1

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,gps,price,dtime,hour,minute,second
0,4.0,william spindler,Best War Wanton soup in Red Bluff,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,1394669496,"Mar 12, 2014",100000032416892623125,"[40.179159, -122.236162]",$$$,2014-03-13 00:11:36,0,11,36
1,5.0,william spindler,This is a review that is long overdo. I've bee...,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,1394826388,"Mar 14, 2014",100000032416892623125,"[40.178074, -122.235234]",$$,2014-03-14 19:46:28,19,46,28
2,5.0,william spindler,"Long time favorite Mexican food, always consis...",['Mexican Restaurant'],115827996910815192564,1394669713,"Mar 12, 2014",100000032416892623125,"[40.175064, -122.242574]",$$,2014-03-13 00:15:13,0,15,13
3,5.0,Anne Mason,Great coffee and location! Check out the used ...,"['Espresso Bar', 'Coffee Shop']",108196901293702895067,1379187049,"Sep 14, 2013",100000106576186066497,"[38.030536, -121.884024]",$$$,2013-09-14 19:30:49,19,30,49
4,5.0,Anne Mason,"I've been here dozens of times, and I'll keep ...","['Sushi Restaurant', 'Asian Restaurant', 'Japa...",111971088396054239194,1394848761,"Mar 14, 2014",100000106576186066497,"[37.961078, -121.748886]",$$,2014-03-15 01:59:21,1,59,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76249,1.0,Richard Wright,This food is disgusting... I am sat in the can...,"['Mexican Restaurant', 'Latin American Restaur...",107540363462823386611,1334257465,"Apr 12, 2012",106390902683872086099,"[33.344747, -118.326901]",$$$,2012-04-12 19:04:25,19,4,25
76250,5.0,Richard Wright,I've eaten Indian food all over the world. A g...,['Indian Restaurant'],112541221653908428450,1360468376,"Feb 9, 2013",106390902683872086099,"[38.74622, -121.260129]",$$,2013-02-10 03:52:56,3,52,56
76251,5.0,Anna Avila,I love this store I can spend hours there and ...,"['Home Improvement Store', 'Hardware Store', '...",100569007418359455686,1347304118,"Sep 10, 2012",106390985705853501877,"[36.717324, -121.664507]",$$$,2012-09-10 19:08:38,19,8,38
76252,3.0,Lizbethh Tovar,This place ive been since I was 5 I love this ...,"['Donut Shop', 'Dessert Shop']",110664528288564115219,1320593088,"Nov 6, 2011",106391014960118925409,"[33.97497, -118.274235]",$$$,2011-11-06 15:24:48,15,24,48


In [2]:
file_path = 'data/places.clean.json.gz'
def read_reviews(file_path):
    with gzip.open(file_path) as file:
        for line_number, line in enumerate(file, start=100000):
            # review = json.loads(eval(line))
            review = eval(line)
            yield review

In [3]:
def find_PlaceId(name):
    file_path = 'data/places.clean.json.gz'
    gplusPlaceId=""
    for review in read_reviews(file_path):
        gplusPlaceName=""
        message_value = json.dumps(review)
        if review["name"] == name:
            gplusPlaceId = review["gPlusPlaceId"]
            break
    return gplusPlaceId

In [4]:
def find_name(place_id):
    file_path = 'data/places.clean.json.gz'
    gplusPlaceName=""
    for review in read_reviews(file_path):
        gplusPlaceName=""
        message_value = json.dumps(review)
        if review["gPlusPlaceId"] == place_id:
            gplusPlaceName = review["name"]
            break
    return gplusPlaceName

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def find_similar_categories(input_category, categories, ratings, reviewer_names, gps):
    # Combine the input category, categories, ratings, and reviewer names into a list
    all_categories = [(input_category, 1.0, "", "")] + list(zip(categories, ratings, reviewer_names, gps))
    
    # Create a TfidfVectorizer
    vectorizer = TfidfVectorizer()
    
    # Transform the categories into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform([category for category, _, _, _ in all_categories])
    
    # Calculate the cosine similarity between the input category and all other categories
    similarity_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])
    
    # Get the indices of the most similar categories
    most_similar_indices = similarity_scores.argsort()[0][::-1]
    
    # Sort the ratings and GPS based on the category indices
    sorted_ratings = sorted(zip(most_similar_indices, ratings, reviewer_names, gps), key=lambda x: x[1], reverse=True)
    sorted_gps = sorted(zip(most_similar_indices, ratings, reviewer_names, gps), key=lambda x: x[3])
    
    unique_categories = set()
    unique_sorted_ratings = []
    
    # Filter out duplicate categories
    for index, rating, reviewer_name, gps_value in sorted_gps:
        category = categories[index]
        if category not in unique_categories:
            unique_categories.add(category)
            unique_sorted_ratings.append((index, rating, reviewer_name, gps_value))
    
    return most_similar_indices[:5], unique_sorted_ratings

### Place ID input 

In [7]:
# Read the data from a CSV file
data = pd.read_csv('df_ca.csv')

# Get the categories, ratings, reviewer names, and GPS columns
categories = data['categories'].tolist()
ratings = data['rating'].tolist()
reviewer_names = data['reviewerName'].tolist()
gps = data['gps'].tolist()
gPlusPlaceId = data['gPlusPlaceId']

# Input gPlusPlaceId
input_gplus_place_id = input("Nhập gPlusPlaceId: ")

# Tìm danh mục (category) tương ứng với input_gplus_place_id
filtered_data = data[data['gPlusPlaceId'] == input_gplus_place_id]
input_category = filtered_data['categories'].iloc[0]

if input_category:
    # Find similar categories and their indices, along with sorted ratings
    similar_category_indices, sorted_ratings = find_similar_categories(input_category, categories, ratings, reviewer_names, gps)

    # Print the similar categories, indices, ratings, reviewer names, and GPS values in the desired format
    print("Indices, Similar Categories, Ratings, Reviewer Names, GPS, Place_ID and Name:")
    for index, rating, reviewer_name, gps_value in sorted_ratings[:5]:
        category = categories[index]
        gplus_place_id = gPlusPlaceId[index]
        place_name = find_name(gplus_place_id)
        print(f"{index:5}  {category:60} {rating:<5} {reviewer_name} {gps_value} {gplus_place_id} {place_name}")
else:
    print("Không tìm thấy gPlusPlaceId trong dữ liệu.")

Nhập gPlusPlaceId: 115827996910815192564
Indices, Similar Categories, Ratings, Reviewer Names, GPS, Place_ID and Name:
53303  ['American Restaurant', 'Breakfast Restaurant', 'Takeout Restaurant'] 4.0   juan manuel [32.494611, -116.933075] 104996079365279085510 IHOP Restaurant
45533  ['Soul Food Restaurant']                                     5.0   Monserrat RetanaContreras [32.494611, -116.933075] 111280162645644452653 Farmerbrown
49440  ['Continental Restaurant', 'Banquet Hall', 'Wine Bar']       5.0   Omar Figueroa [32.494628, -116.95902] 107820498766761981229 The Firehouse Restaurant
33080  ['Sushi Restaurant', 'Asian Restaurant', 'Japanese Restaurant'] 5.0   Carlos Ibarra [32.508423, -116.992893] 101685442908674859812 Fuji Sushi Restaurant
32712  ['Thai Restaurant', 'Asian Restaurant', 'Southeast Asian Restaurant'] 4.0   David Martinez [32.508423, -116.992893] 101789016665204054301 Thai Bite Thai Cuisine
